In [1]:
from fastai2.vision.all import *
import sys
import json, time
sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

from modules.trainutils import piece_class_parse
from modules.trainutils import my_metrics
from modules.trainutils import show_cf, silent_learner
from modules.trainutils import my_test_metrics, my_export_new

In [3]:
test_path = Path('../../../other-chess-data/regulation-test-1')
train_path = Path('../../../rf-chess-data/cropped_v1/')
modelconfig_path = Path('../models/model-config')

In [4]:
_model_arch = resnet50   # to json with resnet50.__name__

_fit_one_cycle_epochs = 10
_fine_tune_epochs = 15

_train_seed =   42   #17

_mult =         1.0
_max_lighting = 0.9  # 0.95
_max_warp    =  0.4  # 0.3
_max_rotate  =  20.  # 30.

# _resize_method = ResizeMethod.Crop
# _resize_method = ResizeMethod.Squish
_resize_method = ResizeMethod.Pad

_pad_mode = 'zeros'  #'reflection'

d = {
    '_model_arch': _model_arch.__name__,
     '_fit_one_cycle_epochs':_fit_one_cycle_epochs,
    '_fine_tune_epochs': _fine_tune_epochs,
    '_train_seed': _train_seed,
    '_mult':_mult,
    '_max_lighting': _max_lighting,
    '_max_warp': _max_warp,
    '_max_rotate': _max_rotate,
    '_resize_method':  _resize_method,
    '_pad_mode':_pad_mode,
}

t0 = time.time()

In [5]:
Crop = Resize(128, _resize_method, pad_mode=_pad_mode)

Augs = aug_transforms(mult=_mult, 
                      max_lighting=_max_lighting,
                      p_lighting=0.75, 
                      max_warp=_max_warp,
                      max_rotate=_max_rotate,
                     )

train_dl = ImageDataLoaders.from_name_func(
                train_path, 
                get_image_files(train_path),
                # stratify_sample(train_path, n=n, np_seed=seed),
                valid_pct=0.2, 
                seed=_train_seed,
                label_func=piece_class_parse, 
                item_tfms=Crop,
                batch_tfms=Augs,
                bs=32,
)

In [6]:
learn = cnn_learner(train_dl, _model_arch, metrics=error_rate)
silent_learner(learn)

In [7]:
learn.fit_one_cycle(_fit_one_cycle_epochs)

In [8]:
test_loss, test_acc = my_test_metrics(learn, test_path)
print(test_loss, test_acc)

model_fn =my_export_new(learn, prefix='expmod-a', print_check=True)

d.update({'test_loss': test_loss, 'test_acc': test_acc})

with open(modelconfig_path / (model_fn.split('.')[0] + '.json'), 'w') as f:
    json.dump(d, f)

0.835702923933665 0.75
found new file expmod-a-0.pkl in ../models


In [9]:
learn.fine_tune(_fine_tune_epochs)

In [10]:
test_loss, test_acc = my_test_metrics(learn, test_path)
print(test_loss, test_acc)

model_fn =my_export_new(learn, prefix='expmod-b', print_check=True)

d.update({'test_loss': test_loss, 'test_acc': test_acc})

with open(modelconfig_path / (model_fn.split('.')[0] + '.json'), 'w') as f:
    json.dump(d, f)

0.807994931936264 0.7833333333333333
found new file expmod-b-0.pkl in ../models


In [11]:
time.time() - t0

305.76691794395447

In [12]:
305 / 60

5.083333333333333